In [1]:
import numpy as np
import random
import math
import multiprocess as mp
import talib as ta
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from datetime import datetime,date
from gga import GGA
from island_ga import IslandGGA 

In [2]:
#rename column to remove spaces in column names
data=pd.read_csv('HistoricalQuotes.csv')
aapl =data.rename(columns={' Close/Last': 'close', ' Volume': 'volume',
                           ' Open':'open',' High':'high',' Low':'low'})
                        

#change date string into date format and sort the dataframe in ascending order
aapl[['Date']] = aapl[['Date']].applymap(lambda x: datetime.strptime(x,'%m/%d/%Y').date())
aapl =aapl.sort_values(by='Date')
#change dataframe dtypes to float
cols = [i for i in aapl if i not in ['Date','volume']]
def to_float(x):
    x = x.strip()
    return float(x[1:])

aapl[cols]=aapl[cols].applymap(to_float)

#Create Signals using Talib library
aapl['5EMA'] = ta.SMA(np.array(aapl['close']),5)
aapl['20EMA'] = ta.EMA(np.array(aapl['close']), timeperiod = 20)
aapl['RSI'] = ta.RSI(np.array(aapl['close']), timeperiod = 14)
aapl['WILLR'] = ta.WILLR(np.array(aapl['high']), np.array(aapl['low']), np.array(aapl['close']), timeperiod=14)
aapl['MOM'] = ta.MOM(np.array(aapl['close']), timeperiod=5)
aapl['CCI'] = ta.CCI(np.array(aapl['high']), np.array(aapl['low']), np.array(aapl['close']), timeperiod=14)
aapl['SLOWK'],aapl['SLOWD'] = ta.STOCH(np.array(aapl['high']), np.array(aapl['low']), np.array(aapl['close']), fastk_period=14, slowk_period=3, slowd_period=3)
aapl['MACD'],aapl['MACDSIGNAL'],aapl['MACDHIST'] = ta.MACD(np.array(aapl['close']), fastperiod=12, slowperiod=26, signalperiod=9)
aapl['DMI'] = ta.DX(np.array(aapl['high']), np.array(aapl['low']), np.array(aapl['close']), timeperiod=14)   

In [3]:
#  """Based on parameter setting adopted in Chen et al (2021) """
def generate_candidate_trading_signals(data):
    """Based on parameter setting adopted in Chen et al (2021) """
    conditions ={'TS1_CONDITIONS':[
                (data['5EMA'] > data['20EMA']),
                (data['5EMA'] < data['20EMA'])],
                 'TS2_CONDITIONS':[
                (data['RSI'] > 30),
                (data['RSI'] < 70),
                ],
                 'TS3_CONDITIONS':[
                (data['WILLR'] < 80),
                (data['WILLR'] > 20),
                ],
                 'TS4_CONDITIONS':[
                (data['MOM'] > 0 ),
                (data['MOM'] <= 0),
                ],
                 'TS5_CONDITIONS': [
                (data['CCI'] > 100 ),
                (data['CCI'] <= 100),
                ],
                 'TS6_CONDITIONS': [
                (data['SLOWK'] > data['SLOWD']) & (data['SLOWD'] < 20),
                (data['SLOWK'] < data['SLOWD']) & (data['SLOWD'] > 80)],
                'TS7_CONDITIONS': [
                (data['MACD'] > 0 ),
                (data['MACD'] <= 0)],
                'TS8_CONDITIONS': [
                (data['CCI'] > 100 ),
                (data['CCI'] <= -100)]}

    # create a list of the values we want to assign for each condition 1: buy, 0: sell
    values = [1, 0]

    # create a new column and use np.select to assign values to it using our lists as arguments
    for i in conditions:
        data[i] = np.select(conditions[i], values)
    strategies =list(conditions.keys())
    return data,strategies

In [14]:
#split data
train = aapl[(aapl['Date'] <= date(2016,12,31))]
test = aapl[(aapl['Date'] > date(2016,12,31))]
train_data, strategies =generate_candidate_trading_signals(train.copy())
test_data, test_strategies =generate_candidate_trading_signals(test.copy())
num_islands=3
num_iter=20
pSize=20
m_iter=5
N=5
K=3
r_cross=0.8
r_mut= 0.01
r_inv=0.3
n=4
b=4
stop_loss=-0.15
take_profit=0.15
allocated_capital=1000
islandgga = IslandGGA(num_islands,num_iter,train_data,strategies,pSize,m_iter,N,K,r_cross,r_mut,r_inv,n,b,stop_loss,take_profit,allocated_capital)

In [72]:
pop =islandgga.init_population()
print(islandgga.strategy_performance(pop[0][0]).var())
islandgga.strategy_performance(pop[0][0]).corr()

TS1_CONDITIONS    0.037359
TS2_CONDITIONS    0.003228
TS3_CONDITIONS    0.003107
TS4_CONDITIONS    0.003413
TS5_CONDITIONS    0.096363
TS6_CONDITIONS    0.254870
TS7_CONDITIONS    0.121211
TS8_CONDITIONS    0.096363
dtype: float64


,TS1_CONDITIONS,TS2_CONDITIONS,TS3_CONDITIONS,TS4_CONDITIONS,TS5_CONDITIONS,TS6_CONDITIONS,TS7_CONDITIONS,TS8_CONDITIONS
TS1_CONDITIONS,1.000000,0.095722,0.138340,0.075313,0.134951,0.000973,0.401021,0.134951
TS2_CONDITIONS,0.095722,1.000000,0.996206,0.933300,-0.305798,0.511374,-0.243473,-0.305798
TS3_CONDITIONS,0.138340,0.996206,1.000000,0.919250,-0.290330,0.499644,-0.228786,-0.290330
TS4_CONDITIONS,0.075313,0.933300,0.919250,1.000000,-0.255579,0.496983,-0.195086,-0.255579
TS5_CONDITIONS,0.134951,-0.305798,-0.290330,-0.255579,1.000000,-0.125876,0.110624,1.000000
TS6_CONDITIONS,0.000973,0.511374,0.499644,0.496983,-0.125876,1.000000,-0.215174,-0.125876
TS7_CONDITIONS,0.401021,-0.243473,-0.228786,-0.195086,0.110624,-0.215174,1.000000,0.110624
TS8_CONDITIONS,0.134951,-0.305798,-0.290330,-0.255579,1.000000,-0.125876,0.110624,1.000000


In [18]:
ring = []
master = []
multi = []
for i in range(50):
    islandgga.evolve_island_ring()
    ring_p4mc = islandgga.evaluate_best_profit(test_data)
    ring.append(islandgga.globalBest)
    islandgga.evolve_master_slave()
    master.append(islandgga.globalBest)
    islandgga.evolve_island_multikuti()
    multi.append(islandgga.globalBest)

In [5]:
islandgga.evolve_island_ring()

Process Process-3:
Process Process-1:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/kuda/Projects/genetic_algos/lib/python3.10/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/kuda/Projects/genetic_algos/lib/python3.10/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/kuda/Projects/genetic_algos/lib/python3.10/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/kuda/Projects/genetic_algos/lib/python3.10/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/kuda/Projects/genetic_algos/lib/python3.10/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/kuda/Projects/genetic_algos/lib/python3.10/site-packages/multiprocess/process.py", line 10

KeyboardInterrupt: 

In [68]:
islandgga.evaluate_best_profit(test_data)

AttributeError: 'IslandGGA' object has no attribute 'evaluate_best_profit'